# **Loading data and Preprocessing**

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d arashnic/hr-analytics-job-change-of-data-scientists

  0% 0.00/295k [00:00<?, ?B/s]
100% 295k/295k [00:00<00:00, 44.6MB/s]


In [ ]:
import os 
import re
import zipfile 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil 
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [ ]:
local_dir='/content/hr-analytics-job-change-of-data-scientists.zip'
zip_ref=zipfile.ZipFile(local_dir,'r')
zip_ref.extractall('/tmp')
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/hr-analytics-job-change-of-data-scientists.zip' mode='r'>>

In [ ]:
df=pd.read_csv("/tmp/aug_train.csv")

In [ ]:
df.columns.values

array(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'], dtype=object)

In [ ]:
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [ ]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [ ]:
df.to_dict()

{'city': {0: 'city_103',
  1: 'city_40',
  2: 'city_21',
  3: 'city_115',
  4: 'city_162',
  5: 'city_176',
  6: 'city_160',
  7: 'city_46',
  8: 'city_103',
  9: 'city_103',
  10: 'city_21',
  11: 'city_103',
  12: 'city_61',
  13: 'city_21',
  14: 'city_21',
  15: 'city_114',
  16: 'city_103',
  17: 'city_21',
  18: 'city_21',
  19: 'city_13',
  20: 'city_159',
  21: 'city_114',
  22: 'city_102',
  23: 'city_40',
  24: 'city_21',
  25: 'city_67',
  26: 'city_21',
  27: 'city_103',
  28: 'city_100',
  29: 'city_21',
  30: 'city_103',
  31: 'city_103',
  32: 'city_103',
  33: 'city_160',
  34: 'city_16',
  35: 'city_103',
  36: 'city_114',
  37: 'city_114',
  38: 'city_103',
  39: 'city_71',
  40: 'city_114',
  41: 'city_104',
  42: 'city_21',
  43: 'city_64',
  44: 'city_21',
  45: 'city_21',
  46: 'city_21',
  47: 'city_103',
  48: 'city_101',
  49: 'city_83',
  50: 'city_101',
  51: 'city_64',
  52: 'city_105',
  53: 'city_104',
  54: 'city_114',
  55: 'city_104',
  56: 'city_73',
 

**Enrollee id**

In [ ]:
df['enrollee_id'].unique()

array([ 8949, 29725, 11561, ..., 24576,  5756, 23834])

In [ ]:
enrollee_id_dict = df['enrollee_id'].to_dict()
enrollee_id_dict

{0: 8949,
 1: 29725,
 2: 11561,
 3: 33241,
 4: 666,
 5: 21651,
 6: 28806,
 7: 402,
 8: 27107,
 9: 699,
 10: 29452,
 11: 23853,
 12: 25619,
 13: 5826,
 14: 8722,
 15: 6588,
 16: 4167,
 17: 5764,
 18: 2156,
 19: 11399,
 20: 31972,
 21: 19061,
 22: 6491,
 23: 7041,
 24: 22767,
 25: 14505,
 26: 17139,
 27: 28476,
 28: 21538,
 29: 10408,
 30: 14928,
 31: 22293,
 32: 4324,
 33: 26966,
 34: 26494,
 35: 4866,
 36: 12726,
 37: 10164,
 38: 8612,
 39: 24659,
 40: 2547,
 41: 13854,
 42: 31654,
 43: 13643,
 44: 5590,
 45: 22452,
 46: 9006,
 47: 25987,
 48: 4476,
 49: 25103,
 50: 5568,
 51: 2195,
 52: 30533,
 53: 28512,
 54: 1023,
 55: 12253,
 56: 25296,
 57: 13238,
 58: 13478,
 59: 18578,
 60: 29975,
 61: 26516,
 62: 24690,
 63: 8433,
 64: 9572,
 65: 5878,
 66: 25695,
 67: 9645,
 68: 12730,
 69: 4830,
 70: 20970,
 71: 17271,
 72: 12731,
 73: 10908,
 74: 29117,
 75: 3686,
 76: 22683,
 77: 22134,
 78: 31765,
 79: 31449,
 80: 5987,
 81: 21762,
 82: 22070,
 83: 25413,
 84: 5902,
 85: 28403,
 86: 30937,

In [ ]:
df['enrollee_id'].value_counts()

2047     1
2692     1
4743     1
27272    1
25225    1
        ..
17682    1
19731    1
29972    1
25878    1
6147     1
Name: enrollee_id, Length: 19158, dtype: int64

In [ ]:
df['enrollee_id'].isna().sum()

0

**city**

In [ ]:
df['city'].unique()

array(['city_103', 'city_40', 'city_21', 'city_115', 'city_162',
       'city_176', 'city_160', 'city_46', 'city_61', 'city_114',
       'city_13', 'city_159', 'city_102', 'city_67', 'city_100',
       'city_16', 'city_71', 'city_104', 'city_64', 'city_101', 'city_83',
       'city_105', 'city_73', 'city_75', 'city_41', 'city_11', 'city_93',
       'city_90', 'city_36', 'city_20', 'city_57', 'city_152', 'city_19',
       'city_65', 'city_74', 'city_173', 'city_136', 'city_98', 'city_97',
       'city_50', 'city_138', 'city_82', 'city_157', 'city_89',
       'city_150', 'city_70', 'city_175', 'city_94', 'city_28', 'city_59',
       'city_165', 'city_145', 'city_142', 'city_26', 'city_12',
       'city_37', 'city_43', 'city_116', 'city_23', 'city_99', 'city_149',
       'city_10', 'city_45', 'city_80', 'city_128', 'city_158',
       'city_123', 'city_7', 'city_72', 'city_106', 'city_143', 'city_78',
       'city_109', 'city_24', 'city_134', 'city_48', 'city_144',
       'city_91', 'city_

In [ ]:
df['city'] = df['city'].replace(
      ['city_103', 'city_40', 'city_21', 'city_115', 'city_162',
       'city_176', 'city_160', 'city_46', 'city_61', 'city_114',
       'city_13', 'city_159', 'city_102', 'city_67', 'city_100',
       'city_16', 'city_71', 'city_104', 'city_64', 'city_101', 'city_83',
       'city_105', 'city_73', 'city_75', 'city_41', 'city_11', 'city_93',
       'city_90', 'city_36', 'city_20', 'city_57', 'city_152', 'city_19',
       'city_65', 'city_74', 'city_173', 'city_136', 'city_98', 'city_97',
       'city_50', 'city_138', 'city_82', 'city_157', 'city_89',
       'city_150', 'city_70', 'city_175', 'city_94', 'city_28', 'city_59',
       'city_165', 'city_145', 'city_142', 'city_26', 'city_12',
       'city_37', 'city_43', 'city_116', 'city_23', 'city_99', 'city_149',
       'city_10', 'city_45', 'city_80', 'city_128', 'city_158',
       'city_123', 'city_7', 'city_72', 'city_106', 'city_143', 'city_78',
       'city_109', 'city_24', 'city_134', 'city_48', 'city_144',
       'city_91', 'city_146', 'city_133', 'city_126', 'city_118',
       'city_9', 'city_167', 'city_27', 'city_84', 'city_54', 'city_39',
       'city_79', 'city_76', 'city_77', 'city_81', 'city_131', 'city_44',
       'city_117', 'city_155', 'city_33', 'city_141', 'city_127',
       'city_62', 'city_53', 'city_25', 'city_2', 'city_69', 'city_120',
       'city_111', 'city_30', 'city_1', 'city_140', 'city_179', 'city_55',
       'city_14', 'city_42', 'city_107', 'city_18', 'city_139',
       'city_180', 'city_166', 'city_121', 'city_129', 'city_8',
       'city_31', 'city_171'],
       
       ['103', '40', '21', '115', '162',
       '176', '160', '46', '61', '114',
       '13', '159', '102', '67', '100',
       '16', '71', '104', '64', '101', '83',
       '105', '73', '75', '41', '11', '93',
       '90', '36', '20', '57', '152', '19',
       '65', '74', '173', '136', '98', '97',
       '50', '138', '82', '157', '89',
       '150', '70', '175', '94', '28', '59',
       '165', '145', '142', '26', '12',
       '37', '43', '116', '23', '99', '149',
       '10', '45', '80', '128', '158',
       '123', '7', '72', '106', '143', '78',
       '109', '24', '134', '48', '144',
       '91', '146', '133', '126', '118',
       '9', '167', '27', '84', '54', '39',
       '79', '76', '77', '81', '131', '44',
       '117', '155', '33', '141', '127',
       '62', '53', '25', '2', '69', '120',
       '111', '30', '1', '140', '179', '55',
       '14', '42', '107', '18', '139',
       '180', '166', '121', '129', '8',
       '31', '171'])
df['city'] = df['city'].astype(int)
df['city']

0        103
1         40
2         21
3        115
4        162
        ... 
19153    173
19154    103
19155    103
19156     65
19157     67
Name: city, Length: 19158, dtype: int64

In [ ]:
city_dict = df['city'].to_dict()
city_dict

{0: 103,
 1: 40,
 2: 21,
 3: 115,
 4: 162,
 5: 176,
 6: 160,
 7: 46,
 8: 103,
 9: 103,
 10: 21,
 11: 103,
 12: 61,
 13: 21,
 14: 21,
 15: 114,
 16: 103,
 17: 21,
 18: 21,
 19: 13,
 20: 159,
 21: 114,
 22: 102,
 23: 40,
 24: 21,
 25: 67,
 26: 21,
 27: 103,
 28: 100,
 29: 21,
 30: 103,
 31: 103,
 32: 103,
 33: 160,
 34: 16,
 35: 103,
 36: 114,
 37: 114,
 38: 103,
 39: 71,
 40: 114,
 41: 104,
 42: 21,
 43: 64,
 44: 21,
 45: 21,
 46: 21,
 47: 103,
 48: 101,
 49: 83,
 50: 101,
 51: 64,
 52: 105,
 53: 104,
 54: 114,
 55: 104,
 56: 73,
 57: 103,
 58: 21,
 59: 162,
 60: 67,
 61: 75,
 62: 41,
 63: 100,
 64: 11,
 65: 93,
 66: 67,
 67: 16,
 68: 16,
 69: 90,
 70: 64,
 71: 21,
 72: 13,
 73: 67,
 74: 11,
 75: 103,
 76: 36,
 77: 103,
 78: 67,
 79: 103,
 80: 103,
 81: 160,
 82: 103,
 83: 103,
 84: 20,
 85: 71,
 86: 57,
 87: 103,
 88: 152,
 89: 103,
 90: 61,
 91: 16,
 92: 103,
 93: 19,
 94: 103,
 95: 65,
 96: 160,
 97: 74,
 98: 65,
 99: 11,
 100: 11,
 101: 173,
 102: 103,
 103: 103,
 104: 16,
 105: 75,

In [ ]:
df['city'].value_counts()

103    4355
21     2702
16     1533
114    1336
160     845
       ... 
129       3
111       3
121       3
171       1
140       1
Name: city, Length: 123, dtype: int64

In [ ]:
df['city'].isna().sum()

0

**city development index**

In [ ]:
df['city_development_index'].unique()

array([0.92 , 0.776, 0.624, 0.789, 0.767, 0.764, 0.762, 0.913, 0.926,
       0.827, 0.843, 0.804, 0.855, 0.887, 0.91 , 0.884, 0.924, 0.666,
       0.558, 0.923, 0.794, 0.754, 0.939, 0.55 , 0.865, 0.698, 0.893,
       0.796, 0.866, 0.682, 0.802, 0.579, 0.878, 0.897, 0.949, 0.925,
       0.896, 0.836, 0.693, 0.769, 0.775, 0.903, 0.555, 0.727, 0.64 ,
       0.516, 0.743, 0.899, 0.915, 0.689, 0.895, 0.89 , 0.847, 0.527,
       0.766, 0.738, 0.647, 0.795, 0.74 , 0.701, 0.493, 0.84 , 0.691,
       0.735, 0.742, 0.479, 0.722, 0.921, 0.848, 0.856, 0.898, 0.83 ,
       0.73 , 0.68 , 0.725, 0.556, 0.448, 0.763, 0.745, 0.645, 0.788,
       0.78 , 0.512, 0.739, 0.563, 0.518, 0.824, 0.487, 0.649, 0.781,
       0.625, 0.807, 0.664])

In [ ]:
df['city_development_index'].to_dict()

{0: 0.92,
 1: 0.7759999999999999,
 2: 0.624,
 3: 0.789,
 4: 0.767,
 5: 0.764,
 6: 0.92,
 7: 0.762,
 8: 0.92,
 9: 0.92,
 10: 0.624,
 11: 0.92,
 12: 0.913,
 13: 0.624,
 14: 0.624,
 15: 0.926,
 16: 0.92,
 17: 0.624,
 18: 0.624,
 19: 0.8270000000000001,
 20: 0.843,
 21: 0.926,
 22: 0.804,
 23: 0.7759999999999999,
 24: 0.624,
 25: 0.855,
 26: 0.624,
 27: 0.92,
 28: 0.887,
 29: 0.624,
 30: 0.92,
 31: 0.92,
 32: 0.92,
 33: 0.92,
 34: 0.91,
 35: 0.92,
 36: 0.926,
 37: 0.926,
 38: 0.92,
 39: 0.884,
 40: 0.926,
 41: 0.924,
 42: 0.624,
 43: 0.6659999999999999,
 44: 0.624,
 45: 0.624,
 46: 0.624,
 47: 0.92,
 48: 0.5579999999999999,
 49: 0.923,
 50: 0.5579999999999999,
 51: 0.6659999999999999,
 52: 0.794,
 53: 0.924,
 54: 0.926,
 55: 0.924,
 56: 0.754,
 57: 0.92,
 58: 0.624,
 59: 0.767,
 60: 0.855,
 61: 0.939,
 62: 0.8270000000000001,
 63: 0.887,
 64: 0.55,
 65: 0.865,
 66: 0.855,
 67: 0.91,
 68: 0.91,
 69: 0.698,
 70: 0.6659999999999999,
 71: 0.624,
 72: 0.8270000000000001,
 73: 0.855,
 74: 0.55,


In [ ]:
df['city_development_index'].value_counts()

0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.649       4
0.807       4
0.781       3
0.625       3
0.664       1
Name: city_development_index, Length: 93, dtype: int64

In [ ]:
df['city_development_index'].isna().sum()

0

**gender sector**

In [ ]:
df['gender'].unique()

array(['Male', nan, 'Female', 'Other'], dtype=object)

In [ ]:
df['gender'] = df['gender'].replace(['Male', 'Female', 'Other', np.NaN],['1', '2', '3', '1.5'])
df['gender'] = df['gender'].astype(float)
df['gender']

0        1.0
1        1.0
2        1.5
3        1.5
4        1.0
        ... 
19153    1.0
19154    1.0
19155    1.0
19156    1.0
19157    1.5
Name: gender, Length: 19158, dtype: float64

In [ ]:
gender_dict = df['gender'].to_dict()
gender_dict

{0: 1.0,
 1: 1.0,
 2: 1.5,
 3: 1.5,
 4: 1.0,
 5: 1.5,
 6: 1.0,
 7: 1.0,
 8: 1.0,
 9: 1.5,
 10: 1.5,
 11: 1.0,
 12: 1.0,
 13: 1.0,
 14: 1.5,
 15: 1.0,
 16: 1.5,
 17: 1.5,
 18: 1.5,
 19: 2.0,
 20: 1.0,
 21: 1.0,
 22: 1.5,
 23: 1.0,
 24: 1.5,
 25: 1.5,
 26: 1.5,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 2.0,
 33: 2.0,
 34: 1.0,
 35: 1.0,
 36: 1.0,
 37: 1.0,
 38: 1.5,
 39: 1.0,
 40: 2.0,
 41: 1.0,
 42: 1.5,
 43: 1.5,
 44: 1.0,
 45: 2.0,
 46: 1.0,
 47: 3.0,
 48: 1.0,
 49: 1.0,
 50: 1.0,
 51: 1.0,
 52: 1.0,
 53: 1.5,
 54: 1.0,
 55: 1.0,
 56: 1.0,
 57: 1.0,
 58: 1.5,
 59: 1.0,
 60: 1.0,
 61: 1.0,
 62: 1.5,
 63: 1.0,
 64: 1.5,
 65: 1.0,
 66: 3.0,
 67: 1.0,
 68: 2.0,
 69: 1.5,
 70: 1.0,
 71: 1.0,
 72: 1.0,
 73: 1.0,
 74: 1.5,
 75: 1.5,
 76: 1.0,
 77: 1.0,
 78: 1.0,
 79: 2.0,
 80: 1.0,
 81: 1.0,
 82: 1.0,
 83: 1.0,
 84: 1.5,
 85: 1.0,
 86: 2.0,
 87: 1.5,
 88: 1.0,
 89: 2.0,
 90: 1.0,
 91: 1.0,
 92: 2.0,
 93: 1.5,
 94: 1.5,
 95: 1.0,
 96: 1.5,
 97: 1.5,
 98: 1.0,
 99: 1.0,
 100: 1.0,

In [ ]:
df['gender'].value_counts()

1.0    13221
1.5     4508
2.0     1238
3.0      191
Name: gender, dtype: int64

In [ ]:
df['gender'].isna().sum()

0

**relevent experience sector**

In [ ]:
df['relevent_experience'].unique()

array(['Has relevent experience', 'No relevent experience'], dtype=object)

In [ ]:
df['relevent_experience'] = df['relevent_experience'].replace(['Has relevent experience', 'No relevent experience'],['1', '0'])
df['relevent_experience'] = df['relevent_experience'].astype(int)
df['relevent_experience']

0        1
1        0
2        0
3        0
4        1
        ..
19153    0
19154    1
19155    1
19156    1
19157    0
Name: relevent_experience, Length: 19158, dtype: int64

In [ ]:
df['relevent_experience'].to_dict()

{0: 1,
 1: 0,
 2: 0,
 3: 0,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 0,
 11: 1,
 12: 1,
 13: 0,
 14: 0,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 0,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 0,
 33: 1,
 34: 1,
 35: 1,
 36: 0,
 37: 1,
 38: 0,
 39: 0,
 40: 1,
 41: 1,
 42: 1,
 43: 0,
 44: 1,
 45: 0,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 0,
 51: 1,
 52: 0,
 53: 1,
 54: 0,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 0,
 65: 1,
 66: 0,
 67: 1,
 68: 1,
 69: 0,
 70: 1,
 71: 0,
 72: 1,
 73: 1,
 74: 0,
 75: 1,
 76: 1,
 77: 0,
 78: 0,
 79: 1,
 80: 1,
 81: 1,
 82: 0,
 83: 1,
 84: 1,
 85: 1,
 86: 0,
 87: 0,
 88: 0,
 89: 1,
 90: 0,
 91: 0,
 92: 0,
 93: 1,
 94: 1,
 95: 1,
 96: 0,
 97: 0,
 98: 1,
 99: 1,
 100: 1,
 101: 0,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 1,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 0,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 12

In [ ]:
df['relevent_experience'].value_counts()

1    13792
0     5366
Name: relevent_experience, dtype: int64

In [ ]:
df['relevent_experience'].isna().sum()

0

**enrolled_university**

In [ ]:
df['enrolled_university'].unique()

array(['no_enrollment', 'Full time course', nan, 'Part time course'],
      dtype=object)

In [ ]:
df['enrolled_university'] = df['enrolled_university'].replace(['no_enrollment', 'Part time course', 'Full time course', np.NaN],['1', '2', '3', '4'])
df['enrolled_university'] = df['enrolled_university'].astype(int)
df['enrolled_university']

0        1
1        1
2        3
3        4
4        1
        ..
19153    1
19154    1
19155    1
19156    1
19157    1
Name: enrolled_university, Length: 19158, dtype: int64

In [ ]:
df['enrolled_university'].to_dict()

{0: 1,
 1: 1,
 2: 3,
 3: 4,
 4: 1,
 5: 2,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 3,
 11: 1,
 12: 1,
 13: 4,
 14: 3,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 2,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 2,
 32: 3,
 33: 1,
 34: 1,
 35: 1,
 36: 2,
 37: 1,
 38: 1,
 39: 1,
 40: 3,
 41: 1,
 42: 1,
 43: 1,
 44: 2,
 45: 3,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 3,
 55: 1,
 56: 3,
 57: 1,
 58: 3,
 59: 1,
 60: 2,
 61: 1,
 62: 4,
 63: 1,
 64: 3,
 65: 1,
 66: 1,
 67: 1,
 68: 1,
 69: 4,
 70: 1,
 71: 3,
 72: 1,
 73: 1,
 74: 3,
 75: 1,
 76: 1,
 77: 1,
 78: 3,
 79: 1,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 2,
 85: 1,
 86: 2,
 87: 3,
 88: 3,
 89: 1,
 90: 3,
 91: 1,
 92: 1,
 93: 1,
 94: 1,
 95: 3,
 96: 3,
 97: 3,
 98: 1,
 99: 1,
 100: 1,
 101: 1,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 1,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 3,
 113: 3,
 114: 1,
 115: 1,
 116: 1,
 117: 3,
 118: 1,
 119: 2,
 120: 1,
 121: 1,
 122: 1,
 12

In [ ]:
df['enrolled_university'].value_counts()

1    13817
3     3757
2     1198
4      386
Name: enrolled_university, dtype: int64

In [ ]:
df['enrolled_university'].isna().sum()

0

**education_level**

In [ ]:
df['education_level'].unique()

array(['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School'],
      dtype=object)

In [ ]:
df['education_level'] = df['education_level'].replace(['Graduate', 'Masters', 'High School', 'Phd', 'Primary School', np.NaN],['1', '2', '3', '4', '5', '3'])
df['education_level'] = df['education_level'].astype(int)
df['education_level']

0        1
1        1
2        1
3        1
4        2
        ..
19153    1
19154    1
19155    1
19156    3
19157    5
Name: education_level, Length: 19158, dtype: int64

In [ ]:
df['education_level'].to_dict()

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 2,
 5: 1,
 6: 3,
 7: 1,
 8: 1,
 9: 1,
 10: 3,
 11: 1,
 12: 1,
 13: 3,
 14: 3,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 2,
 21: 2,
 22: 2,
 23: 1,
 24: 1,
 25: 3,
 26: 1,
 27: 1,
 28: 3,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 3,
 37: 4,
 38: 1,
 39: 3,
 40: 2,
 41: 3,
 42: 2,
 43: 1,
 44: 2,
 45: 2,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 2,
 54: 3,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 2,
 63: 2,
 64: 3,
 65: 1,
 66: 2,
 67: 1,
 68: 2,
 69: 3,
 70: 1,
 71: 1,
 72: 1,
 73: 2,
 74: 1,
 75: 1,
 76: 2,
 77: 1,
 78: 1,
 79: 2,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 2,
 85: 1,
 86: 1,
 87: 3,
 88: 2,
 89: 1,
 90: 1,
 91: 3,
 92: 1,
 93: 1,
 94: 1,
 95: 1,
 96: 3,
 97: 1,
 98: 1,
 99: 1,
 100: 1,
 101: 2,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 2,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 1,
 113: 1,
 114: 2,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 4,
 122: 1,
 12

In [ ]:
df['education_level'].value_counts()

1    11598
2     4361
3     2477
4      414
5      308
Name: education_level, dtype: int64

In [ ]:
df['education_level'].isna().sum()

0

**major_discipline**

In [ ]:
df['major_discipline'].unique()

array(['STEM', 'Business Degree', nan, 'Arts', 'Humanities', 'No Major',
       'Other'], dtype=object)

In [ ]:
df['major_discipline'] = df['major_discipline'].replace(['STEM', 'Business Degree', 'Arts', 'Humanities', 'No Major', 'Other', np.NaN],['1', '2', '3', '4', '5', '6', '3.5'])
df['major_discipline'] = df['major_discipline'].astype(float)
df['major_discipline']

0        1.0
1        1.0
2        1.0
3        2.0
4        1.0
        ... 
19153    4.0
19154    1.0
19155    1.0
19156    3.5
19157    3.5
Name: major_discipline, Length: 19158, dtype: float64

In [ ]:
df['major_discipline'].to_dict()

{0: 1.0,
 1: 1.0,
 2: 1.0,
 3: 2.0,
 4: 1.0,
 5: 1.0,
 6: 3.5,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: 3.5,
 11: 1.0,
 12: 1.0,
 13: 3.5,
 14: 3.5,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 3.0,
 20: 1.0,
 21: 1.0,
 22: 1.0,
 23: 4.0,
 24: 1.0,
 25: 3.5,
 26: 1.0,
 27: 3.0,
 28: 3.5,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 1.0,
 34: 2.0,
 35: 1.0,
 36: 3.5,
 37: 1.0,
 38: 1.0,
 39: 3.5,
 40: 1.0,
 41: 3.5,
 42: 1.0,
 43: 5.0,
 44: 1.0,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 48: 5.0,
 49: 1.0,
 50: 1.0,
 51: 1.0,
 52: 4.0,
 53: 1.0,
 54: 3.5,
 55: 1.0,
 56: 1.0,
 57: 1.0,
 58: 1.0,
 59: 4.0,
 60: 1.0,
 61: 1.0,
 62: 1.0,
 63: 4.0,
 64: 3.5,
 65: 1.0,
 66: 4.0,
 67: 1.0,
 68: 4.0,
 69: 3.5,
 70: 1.0,
 71: 1.0,
 72: 1.0,
 73: 1.0,
 74: 1.0,
 75: 1.0,
 76: 1.0,
 77: 4.0,
 78: 1.0,
 79: 1.0,
 80: 1.0,
 81: 1.0,
 82: 1.0,
 83: 1.0,
 84: 1.0,
 85: 1.0,
 86: 1.0,
 87: 3.5,
 88: 1.0,
 89: 1.0,
 90: 1.0,
 91: 3.5,
 92: 4.0,
 93: 1.0,
 94: 1.0,
 95: 1.0,
 96: 3.5,
 97: 1.0,
 98: 1.0,
 99: 1.0,
 100: 1.0,

In [ ]:
df['major_discipline'].value_counts()

1.0    14492
3.5     2813
4.0      669
6.0      381
2.0      327
3.0      253
5.0      223
Name: major_discipline, dtype: int64

In [ ]:
df['major_discipline'].isna().sum()

0

**experience**

In [ ]:
df['experience'].unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
      dtype=object)

In [ ]:
df['experience'] = df['experience'].replace(['>20', '<1', np.NaN],['21', '0', '11'])
df['experience'] = df['experience'].astype(int)
df['experience']

0        21
1        15
2         5
3         0
4        21
         ..
19153    14
19154    14
19155    21
19156     0
19157     2
Name: experience, Length: 19158, dtype: int64

In [ ]:
df['experience'].to_dict()

{0: 21,
 1: 15,
 2: 5,
 3: 0,
 4: 21,
 5: 11,
 6: 5,
 7: 13,
 8: 7,
 9: 17,
 10: 2,
 11: 5,
 12: 21,
 13: 2,
 14: 5,
 15: 16,
 16: 1,
 17: 2,
 18: 7,
 19: 4,
 20: 11,
 21: 11,
 22: 10,
 23: 0,
 24: 5,
 25: 4,
 26: 14,
 27: 5,
 28: 11,
 29: 18,
 30: 21,
 31: 19,
 32: 5,
 33: 21,
 34: 12,
 35: 10,
 36: 1,
 37: 21,
 38: 12,
 39: 3,
 40: 16,
 41: 4,
 42: 6,
 43: 9,
 44: 9,
 45: 5,
 46: 5,
 47: 19,
 48: 15,
 49: 11,
 50: 4,
 51: 10,
 52: 5,
 53: 3,
 54: 4,
 55: 21,
 56: 2,
 57: 21,
 58: 2,
 59: 21,
 60: 6,
 61: 6,
 62: 13,
 63: 21,
 64: 3,
 65: 10,
 66: 4,
 67: 15,
 68: 14,
 69: 2,
 70: 21,
 71: 0,
 72: 2,
 73: 5,
 74: 0,
 75: 5,
 76: 5,
 77: 3,
 78: 9,
 79: 2,
 80: 15,
 81: 21,
 82: 21,
 83: 5,
 84: 4,
 85: 15,
 86: 0,
 87: 4,
 88: 11,
 89: 6,
 90: 4,
 91: 3,
 92: 3,
 93: 5,
 94: 10,
 95: 9,
 96: 2,
 97: 2,
 98: 6,
 99: 6,
 100: 8,
 101: 21,
 102: 11,
 103: 5,
 104: 12,
 105: 21,
 106: 9,
 107: 2,
 108: 21,
 109: 21,
 110: 8,
 111: 10,
 112: 4,
 113: 2,
 114: 6,
 115: 21,
 116: 3,
 117: 7,

In [ ]:
df['experience'].value_counts()

21    3286
5     1430
4     1403
3     1354
6     1216
2     1127
7     1028
10     985
9      980
8      802
11     729
15     686
14     586
1      549
0      522
16     508
12     494
13     399
17     342
19     304
18     280
20     148
Name: experience, dtype: int64

In [ ]:
df['experience'].isna().sum()

0

**company size**

In [ ]:
df['company_size'].unique()

array([nan, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10/49',
       '100-500', '500-999'], dtype=object)

In [ ]:
df['company_size'] = df['company_size'].replace(['50-99', '<10', '10000+', '5000-9999', '1000-4999', '10/49', '100-500', '500-999', np.NaN],['1', '2', '3','4','5','6','7','8', '4.5'])
df['company_size'] = df['company_size'].astype(float)
df['company_size']

0        4.5
1        1.0
2        4.5
3        4.5
4        1.0
        ... 
19153    4.5
19154    4.5
19155    1.0
19156    8.0
19157    4.5
Name: company_size, Length: 19158, dtype: float64

In [ ]:
df['company_size'].to_dict()

{0: 4.5,
 1: 1.0,
 2: 4.5,
 3: 4.5,
 4: 1.0,
 5: 4.5,
 6: 1.0,
 7: 2.0,
 8: 1.0,
 9: 3.0,
 10: 4.5,
 11: 4.0,
 12: 5.0,
 13: 4.5,
 14: 4.5,
 15: 6.0,
 16: 1.0,
 17: 4.0,
 18: 3.0,
 19: 4.5,
 20: 7.0,
 21: 7.0,
 22: 4.5,
 23: 5.0,
 24: 5.0,
 25: 4.5,
 26: 8.0,
 27: 4.5,
 28: 2.0,
 29: 1.0,
 30: 7.0,
 31: 4.0,
 32: 4.5,
 33: 7.0,
 34: 4.0,
 35: 6.0,
 36: 3.0,
 37: 7.0,
 38: 4.5,
 39: 4.5,
 40: 5.0,
 41: 6.0,
 42: 1.0,
 43: 1.0,
 44: 4.0,
 45: 4.5,
 46: 7.0,
 47: 3.0,
 48: 4.5,
 49: 6.0,
 50: 4.5,
 51: 7.0,
 52: 3.0,
 53: 2.0,
 54: 3.0,
 55: 7.0,
 56: 1.0,
 57: 4.5,
 58: 6.0,
 59: 7.0,
 60: 4.5,
 61: 7.0,
 62: 2.0,
 63: 7.0,
 64: 4.5,
 65: 1.0,
 66: 3.0,
 67: 4.5,
 68: 8.0,
 69: 4.5,
 70: 4.5,
 71: 4.5,
 72: 6.0,
 73: 7.0,
 74: 4.5,
 75: 7.0,
 76: 1.0,
 77: 7.0,
 78: 3.0,
 79: 2.0,
 80: 1.0,
 81: 1.0,
 82: 4.5,
 83: 6.0,
 84: 1.0,
 85: 6.0,
 86: 6.0,
 87: 4.5,
 88: 4.5,
 89: 7.0,
 90: 4.5,
 91: 4.5,
 92: 7.0,
 93: 6.0,
 94: 7.0,
 95: 1.0,
 96: 7.0,
 97: 7.0,
 98: 1.0,
 99: 6.0,
 100: 7.0,

In [ ]:
df['company_size'].value_counts()

4.5    5938
1.0    3083
7.0    2571
3.0    2019
6.0    1471
5.0    1328
2.0    1308
8.0     877
4.0     563
Name: company_size, dtype: int64

In [ ]:
df['company_size'].isna().sum()

0

**company_type**

In [ ]:
df['company_type'].unique()

array([nan, 'Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other',
       'Public Sector', 'NGO'], dtype=object)

In [ ]:
df['company_type'] = df['company_type'].replace(['Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other', 'Public Sector', 'NGO', np.NaN],['1', '2', '3','4','5','6', '3.5'])
df['company_type'] = df['company_type'].astype(float)
df['company_type']

0        3.5
1        1.0
2        3.5
3        1.0
4        2.0
        ... 
19153    3.5
19154    3.5
19155    1.0
19156    1.0
19157    3.5
Name: company_type, Length: 19158, dtype: float64

In [ ]:
df['company_type'].to_dict()

{0: 3.5,
 1: 1.0,
 2: 3.5,
 3: 1.0,
 4: 2.0,
 5: 3.5,
 6: 2.0,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: 3.5,
 11: 1.0,
 12: 1.0,
 13: 3.5,
 14: 3.5,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 3.5,
 20: 1.0,
 21: 1.0,
 22: 3.5,
 23: 1.0,
 24: 1.0,
 25: 3.5,
 26: 1.0,
 27: 3.5,
 28: 1.0,
 29: 2.0,
 30: 1.0,
 31: 1.0,
 32: 3.5,
 33: 1.0,
 34: 1.0,
 35: 3.0,
 36: 4.0,
 37: 1.0,
 38: 3.5,
 39: 3.5,
 40: 5.0,
 41: 1.0,
 42: 3.0,
 43: 1.0,
 44: 1.0,
 45: 3.5,
 46: 1.0,
 47: 5.0,
 48: 3.5,
 49: 1.0,
 50: 3.5,
 51: 1.0,
 52: 1.0,
 53: 3.0,
 54: 1.0,
 55: 1.0,
 56: 3.0,
 57: 3.5,
 58: 2.0,
 59: 1.0,
 60: 3.5,
 61: 1.0,
 62: 3.5,
 63: 1.0,
 64: 3.5,
 65: 1.0,
 66: 1.0,
 67: 3.5,
 68: 5.0,
 69: 1.0,
 70: 3.5,
 71: 3.5,
 72: 1.0,
 73: 3.5,
 74: 3.5,
 75: 1.0,
 76: 1.0,
 77: 1.0,
 78: 1.0,
 79: 3.0,
 80: 1.0,
 81: 1.0,
 82: 3.5,
 83: 3.5,
 84: 2.0,
 85: 3.0,
 86: 1.0,
 87: 3.5,
 88: 5.0,
 89: 3.5,
 90: 3.5,
 91: 3.5,
 92: 1.0,
 93: 1.0,
 94: 6.0,
 95: 1.0,
 96: 1.0,
 97: 1.0,
 98: 1.0,
 99: 1.0,
 100: 1.0,

In [ ]:
df['company_type'].value_counts()

1.0    9817
3.5    6140
2.0    1001
5.0     955
3.0     603
6.0     521
4.0     121
Name: company_type, dtype: int64

In [ ]:
df['company_type'].isna().sum()

0

**last_new_job**

In [ ]:
df['last_new_job'].unique()

array(['1', '>4', 'never', '4', '3', '2', nan], dtype=object)

In [ ]:
df['last_new_job'] = df['last_new_job'].replace(['1', '>4', 'never', 'Other', '4', '3', '2', np.NaN],['1', '2', '0','3','4','5','6', '4'])
df['last_new_job'] = df['last_new_job'].astype(int)
df['last_new_job']

0        1
1        2
2        0
3        0
4        4
        ..
19153    1
19154    4
19155    4
19156    6
19157    1
Name: last_new_job, Length: 19158, dtype: int64

In [ ]:
df['last_new_job'].to_dict()

{0: 1,
 1: 2,
 2: 0,
 3: 0,
 4: 4,
 5: 1,
 6: 1,
 7: 2,
 8: 1,
 9: 2,
 10: 0,
 11: 1,
 12: 5,
 13: 0,
 14: 0,
 15: 2,
 16: 0,
 17: 6,
 18: 0,
 19: 1,
 20: 1,
 21: 6,
 22: 1,
 23: 1,
 24: 1,
 25: 0,
 26: 1,
 27: 6,
 28: 1,
 29: 6,
 30: 5,
 31: 2,
 32: 1,
 33: 2,
 34: 5,
 35: 6,
 36: 1,
 37: 4,
 38: 4,
 39: 0,
 40: 6,
 41: 6,
 42: 1,
 43: 0,
 44: 5,
 45: 0,
 46: 0,
 47: 4,
 48: 2,
 49: 1,
 50: 1,
 51: 6,
 52: 2,
 53: 6,
 54: 1,
 55: 1,
 56: 6,
 57: 6,
 58: 4,
 59: 2,
 60: 1,
 61: 5,
 62: 1,
 63: 2,
 64: 4,
 65: 6,
 66: 4,
 67: 2,
 68: 1,
 69: 0,
 70: 4,
 71: 0,
 72: 1,
 73: 1,
 74: 0,
 75: 6,
 76: 6,
 77: 1,
 78: 5,
 79: 1,
 80: 2,
 81: 6,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 6,
 87: 5,
 88: 6,
 89: 1,
 90: 4,
 91: 0,
 92: 1,
 93: 6,
 94: 1,
 95: 1,
 96: 1,
 97: 1,
 98: 6,
 99: 6,
 100: 1,
 101: 1,
 102: 2,
 103: 1,
 104: 1,
 105: 6,
 106: 6,
 107: 6,
 108: 2,
 109: 4,
 110: 1,
 111: 1,
 112: 1,
 113: 6,
 114: 1,
 115: 2,
 116: 6,
 117: 1,
 118: 4,
 119: 1,
 120: 2,
 121: 6,
 122: 6,
 12

In [ ]:
df['last_new_job'].value_counts()

1    8040
2    3290
6    2900
0    2452
4    1452
5    1024
Name: last_new_job, dtype: int64

In [ ]:
df['last_new_job'].isna().sum()

0

**training_hours**

In [ ]:
df['training_hours'].unique()

array([ 36,  47,  83,  52,   8,  24,  18,  46, 123,  32, 108,  23,  26,
       106,   7, 132,  68,  50,  48,  65,  13,  22, 148,  72,  40, 141,
        82, 145, 206, 152,  42,  14, 112,  87,  20,  21,  92, 102,  43,
        45,  19,  90,  25,  15,  98, 142,  28, 228,  29,  12,  17,  35,
         4, 136,  27,  74,  86,  75, 332, 140, 182, 172,  33,  34, 150,
       160,   3,   2, 210, 101,  59, 260, 131, 109,  70,  51,  60, 164,
       290, 133,  76, 156, 120, 100,  39,  55,  49,   6, 125, 326, 198,
        11,  41, 114, 246,  81,  31,  84, 105,  38, 178, 104, 202,  88,
       218,  62,  10,  80,  77,  37, 162, 190,  30,  16,   5,  54,  44,
       110, 262, 107, 134, 103,  96,  57, 240,  94, 113,  56,  64, 320,
         9, 129,  58, 126, 166,  95,  97, 204, 116, 161, 146, 302,  53,
       143, 124, 214, 288, 306, 322,  67,  61, 130, 220,  78, 314, 226,
       280,  91, 234, 163, 151,  85, 256, 168, 144,  66, 128,  73, 122,
       154,  63, 292, 188,  71, 135, 138, 184,  89, 157, 118, 11

In [ ]:
df['training_hours'].to_dict()

{0: 36,
 1: 47,
 2: 83,
 3: 52,
 4: 8,
 5: 24,
 6: 24,
 7: 18,
 8: 46,
 9: 123,
 10: 32,
 11: 108,
 12: 23,
 13: 24,
 14: 26,
 15: 18,
 16: 106,
 17: 7,
 18: 23,
 19: 132,
 20: 68,
 21: 50,
 22: 48,
 23: 65,
 24: 13,
 25: 22,
 26: 148,
 27: 72,
 28: 8,
 29: 68,
 30: 40,
 31: 141,
 32: 24,
 33: 82,
 34: 145,
 35: 206,
 36: 152,
 37: 42,
 38: 50,
 39: 106,
 40: 14,
 41: 36,
 42: 112,
 43: 108,
 44: 7,
 45: 26,
 46: 87,
 47: 52,
 48: 20,
 49: 21,
 50: 92,
 51: 102,
 52: 26,
 53: 7,
 54: 43,
 55: 45,
 56: 52,
 57: 19,
 58: 32,
 59: 26,
 60: 90,
 61: 25,
 62: 15,
 63: 48,
 64: 98,
 65: 142,
 66: 13,
 67: 47,
 68: 28,
 69: 228,
 70: 22,
 71: 18,
 72: 29,
 73: 46,
 74: 12,
 75: 17,
 76: 65,
 77: 35,
 78: 14,
 79: 4,
 80: 12,
 81: 108,
 82: 136,
 83: 27,
 84: 74,
 85: 86,
 86: 83,
 87: 75,
 88: 42,
 89: 332,
 90: 140,
 91: 182,
 92: 13,
 93: 172,
 94: 48,
 95: 33,
 96: 142,
 97: 34,
 98: 14,
 99: 27,
 100: 24,
 101: 150,
 102: 20,
 103: 140,
 104: 52,
 105: 160,
 106: 3,
 107: 2,
 108: 28,
 10

In [ ]:
df['training_hours'].value_counts()

28     329
12     292
18     291
22     282
50     279
      ... 
266      6
272      5
234      5
286      5
238      4
Name: training_hours, Length: 241, dtype: int64

In [ ]:
df['training_hours'].isna().sum()

0

**target**

In [ ]:
df['target'].unique()

array([1., 0.])

In [ ]:
df['target'].to_dict()

{0: 1.0,
 1: 0.0,
 2: 0.0,
 3: 1.0,
 4: 0.0,
 5: 1.0,
 6: 0.0,
 7: 1.0,
 8: 1.0,
 9: 0.0,
 10: 1.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 1.0,
 19: 1.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 1.0,
 27: 0.0,
 28: 1.0,
 29: 1.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 1.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 1.0,
 43: 0.0,
 44: 1.0,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 48: 1.0,
 49: 0.0,
 50: 1.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 1.0,
 59: 1.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 1.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 1.0,
 70: 0.0,
 71: 1.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 1.0,
 93: 1.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 1.0,
 99: 1.0,
 100: 1.0,

In [ ]:
df['target'].value_counts()

0.0    14381
1.0     4777
Name: target, dtype: int64

In [ ]:
df['target'].isna().sum()

0

**modified df**

In [ ]:
df1 = df
df1

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,103,0.920,1.0,1,1,1,1.0,21,4.5,3.5,1,36,1.0
1,29725,40,0.776,1.0,0,1,1,1.0,15,1.0,1.0,2,47,0.0
2,11561,21,0.624,1.5,0,3,1,1.0,5,4.5,3.5,0,83,0.0
3,33241,115,0.789,1.5,0,4,1,2.0,0,4.5,1.0,0,52,1.0
4,666,162,0.767,1.0,1,1,2,1.0,21,1.0,2.0,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,173,0.878,1.0,0,1,1,4.0,14,4.5,3.5,1,42,1.0
19154,31398,103,0.920,1.0,1,1,1,1.0,14,4.5,3.5,4,52,1.0
19155,24576,103,0.920,1.0,1,1,1,1.0,21,1.0,1.0,4,44,0.0
19156,5756,65,0.802,1.0,1,1,3,3.5,0,8.0,1.0,6,97,0.0


In [ ]:
df1.dtypes

enrollee_id                 int64
city                        int64
city_development_index    float64
gender                    float64
relevent_experience         int64
enrolled_university         int64
education_level             int64
major_discipline          float64
experience                  int64
company_size              float64
company_type              float64
last_new_job                int64
training_hours              int64
target                    float64
dtype: object

In [ ]:
from google.colab import files
df1.to_csv('df1.csv')
files.download('df1.csv')

In [ ]:
from google.colab import  drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
df1.to_csv('/drive/My Drive/41_softCom_project/training_data.csv', index=False)

In [ ]:
df2=pd.read_csv('/drive/My Drive/41_softCom_project/training_data.csv', index_col = False)

In [ ]:
df2.columns.values

array(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'], dtype=object)

In [ ]:
df2

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,103,0.920,1.0,1,1,1,1.0,21,4.5,3.5,1,36,1.0
1,29725,40,0.776,1.0,0,1,1,1.0,15,1.0,1.0,2,47,0.0
2,11561,21,0.624,1.5,0,3,1,1.0,5,4.5,3.5,0,83,0.0
3,33241,115,0.789,1.5,0,4,1,2.0,0,4.5,1.0,0,52,1.0
4,666,162,0.767,1.0,1,1,2,1.0,21,1.0,2.0,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,173,0.878,1.0,0,1,1,4.0,14,4.5,3.5,1,42,1.0
19154,31398,103,0.920,1.0,1,1,1,1.0,14,4.5,3.5,4,52,1.0
19155,24576,103,0.920,1.0,1,1,1,1.0,21,1.0,1.0,4,44,0.0
19156,5756,65,0.802,1.0,1,1,3,3.5,0,8.0,1.0,6,97,0.0


In [ ]:
df2.dtypes

enrollee_id                 int64
city                        int64
city_development_index    float64
gender                    float64
relevent_experience         int64
enrolled_university         int64
education_level             int64
major_discipline          float64
experience                  int64
company_size              float64
company_type              float64
last_new_job                int64
training_hours              int64
target                    float64
dtype: object

In [ ]:
df2['enrollee_id'].unique()

array([ 8949, 29725, 11561, ..., 24576,  5756, 23834])

In [ ]:
df2['city'].unique()

array([103,  40,  21, 115, 162, 176, 160,  46,  61, 114,  13, 159, 102,
        67, 100,  16,  71, 104,  64, 101,  83, 105,  73,  75,  41,  11,
        93,  90,  36,  20,  57, 152,  19,  65,  74, 173, 136,  98,  97,
        50, 138,  82, 157,  89, 150,  70, 175,  94,  28,  59, 165, 145,
       142,  26,  12,  37,  43, 116,  23,  99, 149,  10,  45,  80, 128,
       158, 123,   7,  72, 106, 143,  78, 109,  24, 134,  48, 144,  91,
       146, 133, 126, 118,   9, 167,  27,  84,  54,  39,  79,  76,  77,
        81, 131,  44, 117, 155,  33, 141, 127,  62,  53,  25,   2,  69,
       120, 111,  30,   1, 140, 179,  55,  14,  42, 107,  18, 139, 180,
       166, 121, 129,   8,  31, 171])

In [ ]:
df2['city_development_index'].unique()

array([0.92 , 0.776, 0.624, 0.789, 0.767, 0.764, 0.762, 0.913, 0.926,
       0.827, 0.843, 0.804, 0.855, 0.887, 0.91 , 0.884, 0.924, 0.666,
       0.558, 0.923, 0.794, 0.754, 0.939, 0.55 , 0.865, 0.698, 0.893,
       0.796, 0.866, 0.682, 0.802, 0.579, 0.878, 0.897, 0.949, 0.925,
       0.896, 0.836, 0.693, 0.769, 0.775, 0.903, 0.555, 0.727, 0.64 ,
       0.516, 0.743, 0.899, 0.915, 0.689, 0.895, 0.89 , 0.847, 0.527,
       0.766, 0.738, 0.647, 0.795, 0.74 , 0.701, 0.493, 0.84 , 0.691,
       0.735, 0.742, 0.479, 0.722, 0.921, 0.848, 0.856, 0.898, 0.83 ,
       0.73 , 0.68 , 0.725, 0.556, 0.448, 0.763, 0.745, 0.645, 0.788,
       0.78 , 0.512, 0.739, 0.563, 0.518, 0.824, 0.487, 0.649, 0.781,
       0.625, 0.807, 0.664])

In [ ]:
df2['gender'].unique()

array([1. , 1.5, 2. , 3. ])

In [ ]:
df2['relevent_experience'].unique()

array([1, 0])

In [ ]:
df2['enrolled_university'].unique()

array([1, 3, 4, 2])

In [ ]:
df2['education_level'].unique()

array([1, 2, 3, 4, 5])

In [ ]:
df2['major_discipline'].unique()

array([1. , 2. , 3.5, 3. , 4. , 5. , 6. ])

In [ ]:
df2['experience'].unique()

array([21, 15,  5,  0, 11, 13,  7, 17,  2, 16,  1,  4, 10, 14, 18, 19, 12,
        3,  6,  9,  8, 20])

In [ ]:
df2['company_size'].unique()

array([4.5, 1. , 2. , 3. , 4. , 5. , 6. , 7. , 8. ])

In [ ]:
df2['company_type'].unique()

array([3.5, 1. , 2. , 3. , 4. , 5. , 6. ])

In [ ]:
df2['last_new_job'].unique()

array([1, 2, 0, 4, 5, 6])

In [ ]:
df2['training_hours'].unique()

array([ 36,  47,  83,  52,   8,  24,  18,  46, 123,  32, 108,  23,  26,
       106,   7, 132,  68,  50,  48,  65,  13,  22, 148,  72,  40, 141,
        82, 145, 206, 152,  42,  14, 112,  87,  20,  21,  92, 102,  43,
        45,  19,  90,  25,  15,  98, 142,  28, 228,  29,  12,  17,  35,
         4, 136,  27,  74,  86,  75, 332, 140, 182, 172,  33,  34, 150,
       160,   3,   2, 210, 101,  59, 260, 131, 109,  70,  51,  60, 164,
       290, 133,  76, 156, 120, 100,  39,  55,  49,   6, 125, 326, 198,
        11,  41, 114, 246,  81,  31,  84, 105,  38, 178, 104, 202,  88,
       218,  62,  10,  80,  77,  37, 162, 190,  30,  16,   5,  54,  44,
       110, 262, 107, 134, 103,  96,  57, 240,  94, 113,  56,  64, 320,
         9, 129,  58, 126, 166,  95,  97, 204, 116, 161, 146, 302,  53,
       143, 124, 214, 288, 306, 322,  67,  61, 130, 220,  78, 314, 226,
       280,  91, 234, 163, 151,  85, 256, 168, 144,  66, 128,  73, 122,
       154,  63, 292, 188,  71, 135, 138, 184,  89, 157, 118, 11

In [ ]:
df2['target'].unique()

array([1., 0.])